In [58]:
import numpy as np
import pandas as pd
import re


In [13]:
# for purposes of initial planning, let's go with utterances.csv for now
df1 = pd.read_csv('archive/utterances.csv')
df2 = pd.read_csv('archive/utterances-2sp.csv')

In [14]:
# check columns and shape
df1.columns, df2.columns, df1.shape, df2.shape

(Index(['episode', 'episode_order', 'speaker', 'utterance'], dtype='object'),
 Index(['episode', 'episode_order', 'turn_order', 'speaker_order', 'host_id',
        'is_host', 'utterance'],
       dtype='object'),
 (3199858, 4),
 (1240112, 7))

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199858 entries, 0 to 3199857
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   episode        int64 
 1   episode_order  int64 
 2   speaker        object
 3   utterance      object
dtypes: int64(2), object(2)
memory usage: 97.7+ MB


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240112 entries, 0 to 1240111
Data columns (total 7 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   episode        1240112 non-null  int64 
 1   episode_order  1240112 non-null  int64 
 2   turn_order     1240112 non-null  int64 
 3   speaker_order  1240112 non-null  int64 
 4   host_id        1240112 non-null  int64 
 5   is_host        1240112 non-null  bool  
 6   utterance      1240112 non-null  object
dtypes: bool(1), int64(5), object(1)
memory usage: 58.0+ MB


In [17]:
df1.head()

,episode,episode_order,speaker,utterance
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...","It's a 2,200-mile race. To give some sense of ..."
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",So for a top competitor like Lance to try to m...
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star,..."
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling M...",That's right. Each team has nine riders. And w...
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car ra..."


In [18]:
df2.tail()

,episode,episode_order,turn_order,speaker_order,host_id,is_host,utterance
1240107,141179,21,1,1,-1,False,It's an interesting city and it's so charming ...
1240108,141179,22,0,0,34,False,"Thank you, Ms. Deluca."
1240109,141179,23,0,1,-1,False,Thank you so much.
1240110,141179,24,0,0,34,False,Isabelle Deluca.
1240111,141179,24,1,0,34,False,"She works for the newspaper ""Jornal do Brazil""..."


In [19]:
df1.speaker.value_counts()

NEAL CONAN, host              206098
_NO_SPEAKER                   184423
IRA FLATOW, host               58292
NEAL CONAN, HOST               43533
FARAI CHIDEYA, host            36653
                               ...  
Ms. VANDERMEER                     1
Mr. JIM KITTY (Fire Chief)         1
ROGER IRVING                       1
ROXANNE GORMAN                     1
JOSEPH FISHKIN                     1
Name: speaker, Length: 184963, dtype: int64

In [20]:
df1.episode.nunique()

104920

In [21]:
hosts = {"lulu garcia-navarro": 0, "melissa block": 1, "leila fadel": 2, "susan davis": 3, "sacha pfeiffer": 4, "renee montagne": 5, "korva coleman": 6, "neal conan": 7, "ira flatow": 8, "ari shapiro": 9, "mary louise kelly": 10, "audie cornish": 11, "scott simon": 12, "tony cox": 13, "david greene": 14, "rachel martin": 15, "steve inskeep": 16, "madeleine brand": 17, "alex cohen": 18, "michel martin": 19, "john donvan": 20, "ailsa chang": 21, "lynn neary": 22, "noel king": 23, "flora lichtman": 24, "jennifer ludden": 25, "allison samuels": 26, "sarah mccammon": 27, "alex chadwick": 28, "farai chideya": 29, "celeste headlee": 30, "medeleine brand": 31, "john dankosky": 32, "tom gjelten": 33, "noah adams": 34, "mr. desmond burton (blogger, afro nerd, afro nerd radio on blogtalk radio)": 35, "joe palca": 36, "asma khalid": 37, "greg myre": 38, "faria chideya": 39, "cheryl corley": 40, "alx cohen": 41, "debbie elliott": 42, "jacki lyden": 43, "don gonyea": 44, "lauren frayer": 45, "mr. kevin ross (blogger, three brothers and a sister, the kevin ross show)": 46, "linda wertheimer": 47, "aarti shahani": 48, "a martinez": 49, "scott detrow": 50, "dan charles": 51, "lourdes garcia-navarro": 52, "wade goodwyn": 53, "eric westervelt": 54, "tamara keith": 55, "sean carberry": 56, "jay leno": 57, "faraia chideya": 58, "susan stamberg": 59, "eleanor beardsley": 60, "madeliene brand": 61, "fara chideya": 62, "michele martin": 63, "madeleleine brand": 64, "faraichideya": 65, "don cornelius": 66, "elise hu": 67, "toby": 68, "farai chidey": 69, "brian naylor": 70, "alexchadwick": 71, "madeline brand": 72, "anthony brooks": 73, "an": 74, "rebecca roberts": 75, "carrie kahn": 76, "laura sullivan": 77, "andrea seabrook": 78, "madelein brand": 79, "james hattori": 80, "renee madeleine": 81, "lakshmi singh": 82, "robert smith": 83, "indira lakshmanan": 84, "shereen marisol meraji": 85, "shereen marison meraji": 86, "i": 87, "alec cohen": 88, "deborah amos": 89, "npr n": 90, "ed gordon": 91, "arun rath": 92, "paul raeburn": 93, "npr": 94, "mike huckabee": 95, "neil conan": 96, "noah dams": 97, "john ydstie": 98, "luke burbank": 99, "cokie roberts": 100, "like burbank": 101, "juan williams": 102, "ed gorden": 103, "mike pesca": 104, "robert siegel": 105, "mike presca": 106, "mr. jeff johnson (producer, \"cousin jeff chronicles,\" bet)": 107, "julie mccarthy": 108, "alison stewart": 109, "david welna": 110, "john boehner": 111, "tv": 112, "pbs": 113, "greg allen": 114, "kelly mcevers": 115, "jacques pepin": 116, "liane hansen": 117, "jon j": 118, "a l": 119, "guy raz": 120, "ted koppel": 121, "lynn lyden": 122, "paul shick": 123, "tom goldman": 124, "richard harris": 125, "ray suarez": 126, "mdeleine brand": 127, "p": 128, "david greeene": 129, "mr. alex chadwick": 130, "gwen thompkins": 131, "ferai chideya": 132, "daniel zwerdling": 133, "steve inkseep": 134, "linda werthheimer": 135, "simon scott": 136, "bob edwards": 137, "dwane brown": 138, "stacey vanek smith": 139, "flatow": 140, "michelle martin": 141, "neal cohen": 142, "peter sagal": 143, "tatiana": 144, "andrea seabbrook": 145, "andrew seabrook": 146, "mr. neal conan": 147, "frank stasio": 148, "renee montagnes": 149, "sheila kast": 150, "sheilah kast": 151, "liane hanson": 152, "jackie lyden": 153, "ms. linda wertheimer": 154, "renee motagne": 155, "franks stasio": 156, "mara liasson": 157, "allison aubrey": 158, "david reene": 159, "nathan rott": 160, "cory turner": 161, "tess vigeland": 162, "david green": 163, "karen grigsby bates": 164, "barack obama": 165, "rene montagne": 166, "robet siegel": 167, "audi cornish": 168, "arun rat h": 169, "robert conley": 170, "rachel maddow": 171, "michele norris": 172, "steve inskeeep": 173, "alexandros karamalikis": 174, "rebecca sheir": 175, "scott horsley": 176, "sonari glinton": 177, "steve inskeep and": 178, "peter kenyon": 179, "sylvia poggioli": 180, "frederic wehrey": 181, "h": 182, "mike francessa": 183, "jean dubois": 184, "david folkenflik": 185, "wetheimer": 186, "linda werheimer": 187, "mary louis kelly": 188, "mary louise k": 189, "renee inskeep": 190, "d": 191, "mary louie kelly": 192, "terry gross": 193, "michele brand": 194, "montagne": 195, "sebastian rotella": 196, "michele kelemen": 197, "michel norris": 198, "soraya sarhaddi nelson": 199, "jeff brady": 200, "barbara boxer": 201, "robertr siegel": 202, "jon stewart": 203, "christiane amanpour": 204, "c": 205, "lori healey": 206, "daniel aldrich": 207, "jennifer hunze": 208, "alan cheuse": 209, "richard gonzales": 210, "loren thompson": 211, "kenneth turan": 212, "mike shuster": 213, "reneee montagne": 214, "joost hiltermann": 215, "john mccain": 216, "louisa lim": 217, "steven inskeep": 218, "jackie northam": 219, "ted robbins": 220, "robert sigel": 221, "roberts siegel": 222, "steve isnkeep": 223, "raz": 224, "michele morris": 225, "meliss block": 226, "adam davidson": 227, "nancy youngblood": 228, "rennee montagne": 229, "renee montage": 230, "lucy nalpathanchil": 231, "richard siegel": 232, "madeleine band": 233, "richard lugar": 234, "marylene altieri": 235, "norris": 236, "debra amos": 237, "joseph rich": 238, "rebecce roberts": 239, "rene montage": 240, "rober siegel": 241, "bruce hoffman": 242, "shishir joshi": 243, "john bolton": 244, "robert seigel": 245, "melisa block": 246, "renee montange": 247, "michele norrs": 248, "text  michele norris": 249, "steve ydstie": 250, "julie rovner": 251, "renee montaine": 252, "patricia neighmond": 253, "michelle norris": 254, "mr. noah adams": 255, "ira glass": 256, "renee montaigne": 257, "radek sikorski": 258, "michele block": 259, "steven karpas": 260, "debbie elliot": 261, "melissa blocks": 262, "malowi humdella": 263, "jacki lynden": 264, "anthony kuhn": 265, "wolfgang benz": 266, "michelle block": 267, "mike douglas": 268, "dbbbie elliott": 269, "massimo d": 270, "howard berkes": 271, "paul callenger": 272, "debbie host": 273, "george abed": 274, "jean": 275, "steve gill": 276, "william zinsser": 277, "daniel hernandez": 278, "ann kirschner": 279, "lillian magiltan": 280, "text melissa block": 281, "clip from news broadcast": 282, "kathryn koob": 283, "dwight landry": 284, "kelly mccann": 285, "michele n": 286}

In [9]:
inv_hosts = {v: k for k, v in hosts.items()}
inv_hosts

{0: 'lulu garcia-navarro',
 1: 'melissa block',
 2: 'leila fadel',
 3: 'susan davis',
 4: 'sacha pfeiffer',
 5: 'renee montagne',
 6: 'korva coleman',
 7: 'neal conan',
 8: 'ira flatow',
 9: 'ari shapiro',
 10: 'mary louise kelly',
 11: 'audie cornish',
 12: 'scott simon',
 13: 'tony cox',
 14: 'david greene',
 15: 'rachel martin',
 16: 'steve inskeep',
 17: 'madeleine brand',
 18: 'alex cohen',
 19: 'michel martin',
 20: 'john donvan',
 21: 'ailsa chang',
 22: 'lynn neary',
 23: 'noel king',
 24: 'flora lichtman',
 25: 'jennifer ludden',
 26: 'allison samuels',
 27: 'sarah mccammon',
 28: 'alex chadwick',
 29: 'farai chideya',
 30: 'celeste headlee',
 31: 'medeleine brand',
 32: 'john dankosky',
 33: 'tom gjelten',
 34: 'noah adams',
 35: 'mr. desmond burton (blogger, afro nerd, afro nerd radio on blogtalk radio)',
 36: 'joe palca',
 37: 'asma khalid',
 38: 'greg myre',
 39: 'faria chideya',
 40: 'cheryl corley',
 41: 'alx cohen',
 42: 'debbie elliott',
 43: 'jacki lyden',
 44: 'don g

In [24]:
df2['speaker'] = df2.host_id.map(inv_hosts)
df2

,episode,episode_order,turn_order,speaker_order,host_id,is_host,utterance,speaker
0,1,1,0,0,0,True,The impeachment inquiry picks up tomorrow wher...,lulu garcia-navarro
1,1,1,1,0,0,True,"Just this morning, the lawyer for the whistleb...",lulu garcia-navarro
2,1,1,2,0,0,True,There's are a lot of moving parts.,lulu garcia-navarro
3,1,1,3,0,0,True,"Fortunately, NPR's Mara Liasson is here to help.",lulu garcia-navarro
4,1,2,0,0,0,True,Good morning.,lulu garcia-navarro
...,...,...,...,...,...,...,...,...
1240107,141179,21,1,1,-1,False,It's an interesting city and it's so charming ...,NaN
1240108,141179,22,0,0,34,False,"Thank you, Ms. Deluca.",noah adams
1240109,141179,23,0,1,-1,False,Thank you so much.,NaN
1240110,141179,24,0,0,34,False,Isabelle Deluca.,noah adams


In [26]:
df2[df2.speaker.isnull()]

,episode,episode_order,turn_order,speaker_order,host_id,is_host,utterance,speaker
5,1,3,0,1,-1,False,"Good morning, Lulu.",NaN
8,1,5,0,1,-1,False,"Well, the latest is that the lawyer for the fi...",NaN
9,1,5,1,1,-1,False,The first whistleblower only had second and th...,NaN
10,1,5,2,1,-1,False,We also know that there are subpoenas for Whit...,NaN
11,1,5,3,1,-1,False,We don't know how cooperative the administrati...,NaN
...,...,...,...,...,...,...,...,...
1240102,141179,17,0,1,-1,False,Yes and Jobim is pretty much like Frank Sinatr...,NaN
1240104,141179,19,0,1,-1,False,It's like he's loved by old and the young.,NaN
1240106,141179,21,0,1,-1,False,Maybe because people that's from Rio love Rio ...,NaN
1240107,141179,21,1,1,-1,False,It's an interesting city and it's so charming ...,NaN


In [28]:
df1[df1.episode == 1]

,episode,episode_order,speaker,utterance
2313663,1,1,"LULU GARCIA-NAVARRO, HOST",The impeachment inquiry picks up tomorrow wher...
2313664,1,2,"LULU GARCIA-NAVARRO, HOST",Good morning.
2313665,1,3,"MARA LIASSON, BYLINE","Good morning, Lulu."
2313666,1,4,"LULU GARCIA-NAVARRO, HOST",All right. What's the latest?
2313667,1,5,"MARA LIASSON, BYLINE","Well, the latest is that the lawyer for the fi..."
2313668,1,6,"LULU GARCIA-NAVARRO, HOST","I mean, another whistleblower has got to be ba..."
2313669,1,7,"MARA LIASSON, BYLINE",No. This is kind of like a trail of bread crum...
2313670,1,8,"LULU GARCIA-NAVARRO, HOST",What his happening to his support among Republ...
2313767,1,9,"MARA LIASSON, BYLINE","So far, most Republican lawmakers are shelteri..."
2313768,1,10,"LULU GARCIA-NAVARRO, HOST",That's NPR political correspondent Mara Liasso...


In [29]:
inv_hosts[146]

'andrew seabrook'

In [57]:
re.search(r'\W*(host)\W*', df1.speaker.loc[10])

<re.Match object; span=(10, 16), match=', host'>

In [47]:
df1.speaker.loc[10]

'NEAL CONAN, host'